In [ ]:
!pip install onnxruntime
!pip install onnx
!pip install opencv-python


import onnxruntime as ort
import numpy as np
import cv2 as cv
import math
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')


## Load the ONNX model
model_path = '/content/drive/MyDrive/Computer-Vision/yolo.onnx'
session = ort.InferenceSession(model_path)

## Get model metadata
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

## Setting the anchor dimension array
anchors = [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]

#To label the objects
pascal_classes = [
    "aeroplane",
    "bicycle",
    "bird",
    "boat",
    "bottle",
    "bus",
    "car",
    "cat",
    "chair",
    "cow",
    "dining table",
    "dog",
    "horse",
    "motorbike",
    "person",
    "potted plant",
    "sheep",
    "sofa",
    "train",
    "TV monitor"
]

##Defining some functions to post-process outputs
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def softmax(x):
    e_x = np.exp(x )  ## Subtract max(x) for numerical stability
    return e_x / e_x.sum(axis=0)
#---------

##Using below to remove duplicate object detections

def calculate_iou(box1, box2):      #Intersection Over Union
    # Box format: [x_min, y_min, x_max, y_max]
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    ## Calculating intersection area
    intersection_area = max(0, x2_inter - x1_inter + 1) * max(0, y2_inter - y1_inter + 1)

    ## Calculating areas of both boxes
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    ## Calculating union area
    union_area = box1_area + box2_area - intersection_area

    ## Finally calculating IoU
    iou = intersection_area / union_area

    return iou

## Below also to remove duplicate object detections
def non_max_suppression(boxes, scores, threshold=0.5):
    # List to keep the indices of the selected boxes
    selected_indices = []

    # Creating a list of tuples (score, index) and
    # Sorting it in descending order by score
    score_index_pairs = [(scores[i], i) for i in range(len(scores))]
    score_index_pairs.sort(reverse=True, key=lambda x: x[0])

    # Extract sorted indices based on sorted scores
    sorted_indices = [pair[1] for pair in score_index_pairs]

    while len(sorted_indices) > 0:
        # Take the box with the highest score
        best_idx = sorted_indices[0]
        selected_indices.append(best_idx)

        # Initialize a list to keep the indices of the boxes to keep
        keep_indices = []

        # Compare the highest score box with the rest
        for idx in sorted_indices[1:]:
            iou = calculate_iou(boxes[best_idx], boxes[idx])
            # Only keep boxes with IoU less than the threshold
            if iou < threshold:
                keep_indices.append(idx)

        # Update sorted_indices to only include the boxes to keep
        sorted_indices = keep_indices

    return selected_indices
#---------------------

## Inference loop for multiple images
for i in range(1, 11):
    print(f"Processing image {i}")

    ## Loading image
    image_path = f'/content/drive/MyDrive/Computer-Vision/{i}.jpg'
    image = cv.imread(image_path) ## Default image shape :: (416,416,3)

    ## Preparing input data
    input_data = image.astype(np.float32)
    input_data = np.transpose(input_data, (2, 0, 1)) ## Converted to (3,416,416)
    input_data = np.expand_dims(input_data, axis=0) ## Added one extra dimension (1,3,416,416)

    ## Run inference
    result = session.run([output_name], {input_name: input_data})
    predictions = result[0] ## As taking 1st batch's result

    ## Initializing to extract post-process good predictions
    boxes = []
    scores = []
    confi=[]
    names=[]
    #-----------

    ## Processing the output to print through OpenCv
    for row in range(13):
        for col in range(13):

            ## First getting the highest confidence bounding box from total 5
            ## for each Grid cell
            confidence_probablity = {}
            for j in range(5):

                ## Getting the outputs
                tx = predictions[0][25*j][row][col]
                ty = predictions[0][1 + 25*j][row][col]
                tw = predictions[0][2 + 25*j][row][col]
                th = predictions[0][3 + 25*j][row][col]
                tc = predictions[0][4 + 25*j][row][col]

                ##Processing the outputs
                confidence = sigmoid(tc)
                center_x = (col + sigmoid(tx)) * 32
                center_y = (row + sigmoid(ty)) * 32

                ## Below, anchor[2*j]*32 because anchor values is relative to grid dimension
                width = math.exp(tw) *(anchors[2*j] * 32)
                height = math.exp(th) *(anchors[2*j + 1] * 32)

                prob_list = [1]*20
                for k in range(20):
                    prob_list[k]=predictions[0,k + (j*25+5), row, col]
                    prob_list = softmax(prob_list)
                    confidence_probablity[confidence] = prob_list.tolist()
                #----------------

            ##Now,getting the highest confidence one
            sorted_confidences = sorted(confidence_probablity.keys(), reverse=True)
            highest_confidence = sorted_confidences[0] #
            best_probabilities = confidence_probablity[highest_confidence]
            best_probability = max(best_probabilities)
            best_probability_index = best_probabilities.index(best_probability)
            obj_name=pascal_classes[best_probability_index]

            #Extracting high confidence cell points
            if (highest_confidence > 0.35):  ## Adjusting  threshold as needed

                ##Final calculation of boundary box
                x_min = int(center_x - width / 2)
                y_min = int(center_y - height / 2)
                x_max = int(center_x + width / 2)
                y_max = int(center_y + height / 2)

                ## Storing the good data
                boxes.append([x_min, y_min, x_max, y_max])
                scores.append(highest_confidence)
                confi.append(highest_confidence)
                names.append(obj_name)

    ## Convert to numpy arrays
    boxes = np.array(boxes)
    scores = np.array(scores)

    ## Apply Non-Maximum Suppression
    selected_indices = non_max_suppression(boxes, scores, threshold=0.35)

    ## Drawing finally selected boxes after NMS
    for idx in selected_indices:
        x_min, y_min, x_max, y_max = boxes[idx]
        cv.rectangle(image, (x_min, y_min), (x_max, y_max), (0,0 ,255 ), 2)
        cv.putText(image, f"Conf :: {confi[idx]:.2f} || {names[idx]}", (x_min, y_min - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

    ## Show the image with detections
     ## Display the image with detections using matplotlib
    plt.figure(figsize=(8, 6))
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f"Detected Objects - Image {i}")
    plt.show()
